In [95]:
import sys
sys.path.append('app/src')
sys.path.append('./src')
from zono_sparse_gen import ZonoSparseGeneration
test_input = torch.randn(3,224,224)
_,zonotope_espilon_sparse_tensor = ZonoSparseGeneration(test_input,0.01).total_zono()

/home/guillaume/Documents/AbstratRay/app/./src/zono_sparse_gen.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(tensor)


In [114]:
import sys
sys.path.append('app/src')
sys.path.append('./src')
from zono_sparse_gen import ZonoSparseGeneration
test_input = torch.randn(3,224,224)
_,zonotope_espilon_sparse_tensor = ZonoSparseGeneration(test_input,0.01).total_zono()

print(zonotope_espilon_sparse_tensor)


tensor(indices=tensor([[     0,      1,      2,  ..., 150525, 150526, 150527],
                       [     0,      0,      0,  ...,      2,      2,      2],
                       [     0,      0,      0,  ...,    223,    223,    223],
                       [     0,      1,      2,  ...,    221,    222,    223]]),
       values=tensor([0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]),
       size=(150528, 3, 224, 224), nnz=150528, layout=torch.sparse_coo)


/home/guillaume/Documents/AbstratRay/app/./src/zono_sparse_gen.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(tensor)


In [143]:
import torch
import torch.nn.functional as F

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, dilation =1, bias=True, weights=None, bias_val=None):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.groups = groups
        self.dilation = dilation
        
        # Initialize weights and biases
        if weights is None:
            self.weights = torch.randn(out_channels, in_channels // groups, kernel_size, kernel_size)
        else:
            self.weights = torch.tensor(weights, dtype=torch.float32)
        
        if bias:
            if bias_val is None:
                self.bias = torch.randn(out_channels)
            else:
                self.bias = torch.tensor(bias_val, dtype=torch.float32)
        else:
            self.bias = None
        

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, H, W)
        Returns:
            The result of the convolution as a sparse tensor in COO format
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()
        
        B, C, H, W = sparse_tensor.shape
        out_height = (H + 2 * self.padding - self.dilation*(self.kernel_size-1)-1) // self.stride + 1
        out_width = (W + 2 * self.padding - self.dilation *(self.kernel_size-1)-1) // self.stride + 1
        output_values = []
        output_indices = []
        


        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):

            b, c, h, w = indices[:, i].tolist()
            value = values[i].item()
           
            for kh in range(self.kernel_size):
                for kw in range(self.kernel_size):
                    h_out = ((h + self.padding - kh * self.dilation)+1) // self.stride -1
                    w_out = ((w + self.padding - kw * self.dilation)+1) // self.stride -1
                    if 0 <= h_out < out_height and 0 <= w_out < out_width:
                        for k in range(self.weights.shape[0]):
                        
                            output_values.append(value * self.weights[k, (c % (C // self.groups)), kh, kw].item())
                            output_indices.append([b, k, h_out, w_out])

        # Apply bias
        if self.bias is not None:
            for b in range(B):
                for k in range(self.weights.shape[0]):
                    for h_out in range(out_height):
                        for w_out in range(out_width):
                            output_values.append(self.bias[k].item())
                            output_indices.append([b, k, h_out, w_out])

        output_values = torch.tensor(output_values)
        output_indices = torch.tensor(output_indices).T
        
        size = (B, self.weights.shape[0], out_height, out_width)
        sparse_output = torch.sparse_coo_tensor(output_indices, output_values, size)
        
        return sparse_output

def conv2d_to_sparseconv2d(conv2d):
    """
    Transforms a torch.nn.Conv2d instance to a SparseConv2D instance
    
    Args:
        conv2d: instance of torch.nn.Conv2d
    
    Returns:
        instance of SparseConv2D
    """
    in_channels = conv2d.in_channels
    out_channels = conv2d.out_channels
    kernel_size = conv2d.kernel_size[0]
    stride = conv2d.stride[0]
    padding = conv2d.padding[0]
    groups = conv2d.groups
    weights = conv2d.weight.detach().numpy()
    dilation = conv2d.dilation[0]
    if conv2d.bias is not None:
        bias = conv2d.bias.detach().numpy()
    else:
        bias = None
    
    return SparseConv2D(in_channels, out_channels, kernel_size, stride, padding, groups=groups, dilation =dilation,bias=(bias is not None), weights=weights, bias_val=bias)

# Example usage:
conv2d = torch.nn.Conv2d(in_channels=3, out_channels=30, kernel_size=3, stride=1, padding=0, groups=1,dilation=1)

conv2d.bias = None
sparse_conv2d = conv2d_to_sparseconv2d(conv2d)

# Creating a sparse tensor for testing
B, C, H, W = 3, 3, 10, 10
indices = torch.tensor([[0, 0, 0, 0], [0, 0, 2, 0], [2, 1, 3, 0], [2, 1, 3, 0]])
values = torch.tensor([1., 2., 3., 4.])
data = torch.sparse_coo_tensor(indices, values, size=[B, C, H, W])

# Applying the sparse convolution
result = sparse_conv2d(data)

# For comparison, dense convolution on a dense version of the sparse tensor
dense_data = data.to_dense()
conv2d_result = conv2d(dense_data)



print(torch.sum(conv2d_result-result.to_dense()))

tensor(-2.1048e-07, grad_fn=<SumBackward0>)


In [115]:
result_2 =sparse_conv2d(zonotope_espilon_sparse_tensor)

KeyboardInterrupt: 

In [104]:
result_2

tensor(indices=tensor([[    0,     0,     0,  ..., 37631, 37631, 37631],
                       [    0,     1,     2,  ...,     0,     1,     2],
                       [    0,     0,     0,  ...,   109,   109,   109],
                       [    0,     0,     0,  ...,   109,   109,   109]]),
       values=tensor([-0.0001,  0.0010,  0.0004,  ..., -0.0006, -0.0013,
                       0.0006]),
       size=(37632, 3, 110, 110), nnz=980100, layout=torch.sparse_coo)

In [151]:
import torch
import torch.nn.functional as F

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, dilation=1, bias=True, weights=None, bias_val=None):
        self.kernel_size = kernel_size
        self.stride = stride
        print(self.stride)
        self.padding = padding
        self.groups = groups
        self.dilation = dilation

        # Initialize weights and biases
        if weights is None:
            self.weights = torch.randn(out_channels, in_channels // groups, kernel_size, kernel_size)
        else:
            self.weights = torch.tensor(weights, dtype=torch.float32)

        if bias:
            if bias_val is None:
                self.bias = torch.randn(out_channels)
            else:
                self.bias = torch.tensor(bias_val, dtype=torch.float32)
        else:
            self.bias = None

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, H, W)
        Returns:
            The result of the convolution as a sparse tensor in COO format
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()

        B, C, H, W = sparse_tensor.shape
        out_height = (H + 2 * self.padding - self.dilation * (self.kernel_size - 1) - 1) // self.stride + 1
        out_width = (W + 2 * self.padding - self.dilation * (self.kernel_size - 1) - 1) // self.stride + 1
        
        output = {
            'values': [],
            'indices': []
        }

        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):
            b, c, h, w = indices[:, i].tolist()
            value = values[i].item()

            for kh in range(self.kernel_size):
                for kw in range(self.kernel_size):
                    h_out = (h + self.padding - kh * self.dilation) // self.stride
                    w_out = (w + self.padding - kw * self.dilation) // self.stride
                    

                    if 0 <= h_out < out_height and 0 <= w_out < out_width:
                        for k in range(self.weights.shape[0]):
                            group_idx = c // (C // self.groups)
                            weight_idx = c % (C // self.groups)
                            conv_value = value * self.weights[k, weight_idx, kh, kw].item()

                            output['values'].append(conv_value)
                            output['indices'].append([b, k, h_out, w_out])

        # Apply bias
        if self.bias is not None:
            for b in range(B):
                for k in range(self.weights.shape[0]):
                    for h_out in range(out_height):
                        for w_out in range(out_width):
                            output['values'].append(self.bias[k].item())
                            output['indices'].append([b, k, h_out, w_out])

        output['values'] = torch.tensor(output['values'])
        output['indices'] = torch.tensor(output['indices']).T
        print(output['indices'])

        size = (B, self.weights.shape[0], out_height, out_width)
        sparse_output = torch.sparse_coo_tensor(output['indices'], output['values'], size)

        return sparse_output,torch.sum(torch.abs(sparse_output),dim=0)

def conv2d_to_sparseconv2d(conv2d):
    """
    Transforms a torch.nn.Conv2d instance to a SparseConv2D instance
    
    Args:
        conv2d: instance of torch.nn.Conv2d
    
    Returns:
        instance of SparseConv2D
    """
    in_channels = conv2d.in_channels
    out_channels = conv2d.out_channels
    kernel_size = conv2d.kernel_size[0]
    stride = conv2d.stride[0]
    padding = conv2d.padding[0]
    groups = conv2d.groups
    weights = conv2d.weight.detach().numpy()
    dilation = conv2d.dilation[0]
    if conv2d.bias is not None:
        bias = conv2d.bias.detach().numpy()
    else:
        bias = None

    return SparseConv2D(in_channels, out_channels, kernel_size, stride, padding, groups=groups, dilation=dilation, bias=(bias is not None), weights=weights, bias_val=bias)
# Example usage:
conv2d = torch.nn.Conv2d(in_channels=3, out_channels=30, kernel_size=3, stride=1
                         , padding=0, groups=1,dilation=1)

conv2d.bias = None
sparse_conv2d = conv2d_to_sparseconv2d(conv2d)

# Creating a sparse tensor for testing
B, C, H, W = 3, 3, 50, 50
indices = torch.tensor([[0, 0, 0, 0], [0, 0, 2, 2], [2, 1, 3, 0], [2, 1,25, 25]])
values = torch.tensor([1., 2., 3., 4.])
data = torch.sparse_coo_tensor(indices, values, size=[B, C, H, W])

# Applying the sparse convolution
result,sum = sparse_conv2d(data)

# For comparison, dense convolution on a dense version of the sparse tensor
dense_data = data.to_dense()
conv2d_result = conv2d(dense_data)



print(torch.sum(conv2d_result-result.to_dense()))

1
tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  1,  2,  ..., 27, 28, 29],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ..., 23, 23, 23]])
tensor(0., grad_fn=<SumBackward0>)


In [152]:
sum

tensor(indices=tensor([[ 0,  0,  0,  ..., 29, 29, 29],
                       [ 0,  0,  0,  ...,  3,  3,  3],
                       [ 0,  1,  2,  ..., 23, 24, 25]]),
       values=tensor([1.3801e-01, 2.0934e-01, 2.9606e-02, 5.0667e-02,
                      3.2831e-01, 1.4538e-01, 1.7595e-01, 3.3262e-01,
                      1.1319e-01, 2.2810e-01, 2.2187e-03, 1.6033e-01,
                      6.7272e-02, 8.8782e-02, 1.6036e-01, 3.5208e-01,
                      4.1958e-01, 5.7243e-01, 3.8000e-02, 2.4624e-01,
                      1.0904e-01, 1.7704e-01, 1.3664e-01, 1.3593e-01,
                      5.6017e-01, 1.9029e-01, 4.1737e-01, 1.4186e-01,
                      1.2469e-01, 7.6905e-02, 2.7034e-01, 3.1433e-01,
                      9.9045e-02, 1.2914e-01, 4.5410e-02, 6.8154e-02,
                      5.3082e-01, 4.9543e-01, 4.1238e-02, 4.2012e-01,
                      1.4271e-01, 3.1303e-01, 1.4734e-01, 3.4150e-01,
                      1.1239e-01, 4.8653e-01, 7.8020e-02, 6.822

In [144]:
import sys
sys.path.append('app/src')
sys.path.append('./src')
from zono_sparse_gen import ZonoSparseGeneration
test_input = torch.randn(3,112,112)
_,zonotope_espilon_sparse_tensor = ZonoSparseGeneration(test_input,0.01).total_zono()

print(zonotope_espilon_sparse_tensor)

tensor(indices=tensor([[    0,     1,     2,  ..., 37629, 37630, 37631],
                       [    0,     0,     0,  ...,     2,     2,     2],
                       [    0,     0,     0,  ...,   111,   111,   111],
                       [    0,     1,     2,  ...,   109,   110,   111]]),
       values=tensor([0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]),
       size=(37632, 3, 112, 112), nnz=37632, layout=torch.sparse_coo)


In [147]:
result_2 =sparse_conv2d(zonotope_espilon_sparse_tensor)


tensor([[    0,     0,     0,  ..., 37631, 37631, 37631],
        [    0,     1,     2,  ...,    27,    28,    29],
        [    0,     0,     0,  ...,   109,   109,   109],
        [    0,     0,     0,  ...,   109,   109,   109]])


In [148]:
result_2

tensor(indices=tensor([[    0,     0,     0,  ..., 37631, 37631, 37631],
                       [    0,     1,     2,  ...,    27,    28,    29],
                       [    0,     0,     0,  ...,   109,   109,   109],
                       [    0,     0,     0,  ...,   109,   109,   109]]),
       values=tensor([ 4.8458e-04, -1.7757e-03, -1.7573e-03,  ...,
                      -3.7899e-04, -8.3448e-05, -6.4490e-04]),
       size=(37632, 30, 110, 110), nnz=9801000, layout=torch.sparse_coo)